utiles d'analyse pout TP MMC UGA 2025

lien ver les TP en format Markdown:

https://github.com/xu-uga-2022/MMC-TP-2025

Repo Public, please create a github account to copy and modify files. Thank you very much

#### Tension superficielle



Essayer d'utiliser Python pour analyser automatiquement les données de mesure.

en fin, renouveaulez les constant pour le mesure differents

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Définition des constantes
g = 9.81  # accélération gravitationnelle (m/s²)
rho_water = 998.0  # densité de l'eau (kg/m³), température supposée 20°C
rho_alcohol = 789.0  # densité de l'alcool (kg/m³), supposé éthanol



il faut note les resultats des mesures dans la forme suivante, dans un fichier .csv

```
diameter_mm,mass_g
1.0,0.035
1.5,0.052
2.0,0.070
2.5,0.087
```


In [ ]:
# 读取CSV数据（输入文件格式：'diameter_mm', 'mass_g'）
data = pd.read_csv('droplet_data.csv')

# 单独的实验：例如利用直径介于0.5mm至2mm的注射器滴出50滴水
# 单位转换：直径(mm) → 半径(m), 质量(g) → 重量(N)
data['radius_m'] = data['diameter_mm'] * 1e-3 / 2  # 注射器半径
data['weight_N'] = data['mass_g'] * 1e-3 * g  # 滴液重量 (N)

# 根据Tate定律计算表面张力 (γ = 重量 / (2πr))
data['gamma'] = data['weight_N'] / (2 * np.pi * data['radius_m'])

# 输出计算结果
print("计算得到的表面张力：")
print(data[['diameter_mm', 'gamma']])




In [ ]:

# Visualisation
plt.figure(figsize=(10, 5))
plt.plot(data['diameter_mm'], data['gamma'], 'bo-', label='滴液')
plt.xlabel('diametre (mm)')
plt.ylabel('tension superficielle (N/m)')
plt.title('Tension superficielle par la méthode d’arra-chement')
plt.legend()
plt.grid()
plt.show()

# correction de la loi de Tate, avec un coefficient C
def tate_law(r, gamma, C):
    return C * 2 * np.pi * r * gamma / g  # 修正后的Tate定律

# curve_fit() fonction, pour ajuster les paramètres de la loi de Tate
popt, _ = curve_fit(tate_law, data['radius_m'], data['mass_g'] * 1e-3)
print(f"tension superficielle apres correction: {popt[0]:.4f} N/m, coefficient C = {popt[1]:.2f}")

In [ ]:
from sklearn.metrics import mean_squared_error

# 使用拟合参数预测质量（单位：kg）
predicted_mass_kg = tate_law(data['radius_m'], *popt)

# 实际质量（单位：kg）
actual_mass_kg = data['mass_g'] * 1e-3

# 计算 MSE（均方误差）
mse = mean_squared_error(actual_mass_kg, predicted_mass_kg)

print(f"mean square error (MSE): {mse:.6e} kg²")


### Loi de Poisueill

utilize une document .csv pour note les resultats.

example:
```
1.0
0.8
0.5
0.3
......
```

In [ ]:
# read data
poisueill_data = pd.read_csv('poisueill_data.csv', header=None, names=['height_cm'])
# transvers l'attitude a difference de pression
poisueill_data['pressure_Pa'] = rho_water * g * (poisueill_data['height_cm'] / 100)
# 计算相邻管之间的高度差及对应的压降差
poisueill_data['height_diff_cm'] = poisueill_data['height_cm'].diff()
poisueill_data['pressure_diff_Pa'] = poisueill_data['pressure_Pa'].diff()
# Distance entre les tubes verticaux en METTRE
tube_spacing_m = 0.05
poisueill_data['pressure_gradient_Pa_per_m'] = poisueill_data['pressure_diff_Pa'] / tube_spacing_m
# 水流量数据：假设每分钟水流量为5升，将其转换为 m³/s
flow_rate_lpm = 5
# volume flow rate (Litre/minute) → flow rate (m³/s)
flow_rate_m3_s = (flow_rate_lpm / 1000) / 60
# output
print("Poisueill experiment: Hateur dans les tube verticaux et pertes de charges：")
print(poisueill_data)
print(f"Flow：{flow_rate_m3_s:.6f} m³/s")

### Seuil en V

